In [17]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd '/content/drive/My Drive/Colab_Notebooks_Music_Gen'
%ls

/content/drive/My Drive/Colab_Notebooks_Music_Gen
data/       lstm.py      new_weights.hdf5  predict.py
lstm.ipynb  midi_songs/  predict.ipynb


In [0]:
def train_network():
    """ Train a Neural Network to generate music """
    
    # get list of notes and normalOrder notation for chords
    notes = get_notes()
    
    # get amount of pitch names
    # set indicates no duplicates
    n_vocab = len(set(notes))
     
    # get input and output for network
    network_input, network_output = prepare_sequences(notes, n_vocab)

    # create the model for training
    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [0]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    
    # empty list to contain all notes
    notes = []
    
    # get all files in midi_songs directory in the form of
    # "midi_songs/*.mid"
    for file in glob.glob("midi_songs/*.mid"):
      
        # from file get Score produced from parse function
        midi = converter.parse(file)
  
        # notify which file is being parsed
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            # partition the midi file by instruments, return that list
            # into s2
            s2 = instrument.partitionByInstrument(midi)
            
            # parses first part of midi 
            # recurse() will visit every element in the stream, 
            # starting from the beginning, and if any of the 
            # subelements are also Streams, they will visit 
            # every element in that Stream.
            notes_to_parse = s2.parts[0].recurse() 
            
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        # loop through elements in notes_to_parse
        for element in notes_to_parse:
            # is element a note object?
            if isinstance(element, note.Note):
                # if so append the pitch (note) to the notes list
                notes.append(str(element.pitch))
            # is element a chord object?
            elif isinstance(element, chord.Chord):
                # if so append the chord to the notes list by joining
                # each element in normalOrder list of integer representation
                notes.append('.'.join(str(n) for n in element.normalOrder))
    
    # open 'data/notes' file for writing in binary format since we are
    # dealing with non text format
    with open('data/notes', 'wb') as filepath:
        # write notes in binary format to filepath
        pickle.dump(notes, filepath)
    # return notes list
    return notes

In [0]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # define a sequence length, can be any length
    sequence_length = 100

    # get all unique pitch names in sorted order
    # including notes and normalOrder chords
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
  
    # create empty lists for input and output
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    # go from 0 to len(notes) - sequence_length in increments of 1
    for i in range(0, len(notes) - sequence_length, 1):
        # sequence_in is a sequence of length 100 
        # going from i all the way to one before i + sequence length
        # in notes list
        sequence_in = notes[i:i + sequence_length]
        # sequence out is a single note from notes list
        # that is at position i + sequence length
        # right after the last note in sequence_in 
        sequence_out = notes[i + sequence_length]
        
        # append all int representations of notes or chords from sequence_n
        network_input.append([note_to_int[char] for char in sequence_in])
        # append int representation of note or chord from sequence_out
        network_output.append(note_to_int[sequence_out])

    # number of patterns is eq to length of network input
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # converts a class vector (integers) to binary class matrix.
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [0]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

In [0]:
if __name__ == '__main__':
    train_network()

Parsing midi_songs/0fithos.mid
Parsing midi_songs/8.mid
Parsing midi_songs/ahead_on_our_way_piano.mid
Parsing midi_songs/AT.mid
Epoch 1/200
1792/2161 [=======================>......] - ETA: 4s - loss: 4.4986